# Bayesian Optimization for Ancient Scenarios

 Bayesian Optimization for ancient selection scenarios using the following parameters:
- **Number of Trials:** 30
- **Number of Simulations Used:** 40,000
- **GPU Used:** A100


In [1]:
!pip install optuna
!pip install optuna-integration



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 3.9 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/Djinho/EvoNet-CNN-Insight.git

# Navigate to the cloned repository
%cd EvoNet-CNN-Insight/Optimisation_notebooks

Cloning into 'EvoNet-CNN-Insight'...
remote: Enumerating objects: 5675, done.
remote: Counting objects: 100% (1296/1296), done.
remote: Compressing objects: 100% (938/938), done.
remote: Total 5675 (delta 464), reused 1112 (delta 341), pack-reused 4379
Receiving objects: 100% (5675/5675), 8.07 GiB | 16.99 MiB/s, done.
Resolving deltas: 100% (2062/2062), done.
Updating files: 100% (264/264), done.
/content/EvoNet-CNN-Insight/Optimisation_notebooks


In [3]:
%run -i ../ImaGene.py

In [5]:
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import logging
import os
import time
import gzip
import skimage.transform
from optuna.integration import TFKerasPruningCallback

# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AM.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_moderate/AM'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///am_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=30)  # Set to 30 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function



Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_moderate/AM


[I 2024-08-04 13:36:59,143] A new study created in RDB with name: no-name-27a1444c-e844-4903-abb6-66718ee507fa


Trial 0 started with hyperparameters: num_layers=3, filters=[81, 126, 62], kernel_size=2, l1=0.003188315471455216, l2=2.1306421274251034e-06, dense_units=188, dropout_rate=0.2436894556788965, learning_rate=1.5036844388351935e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - accuracy: 0.5079 - loss: 10.6863 - val_accuracy: 0.4825 - val_loss: 10.5606
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5231 - loss: 10.5152 - val_accuracy: 0.4837 - val_loss: 10.3877
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5693 - loss: 10.3430 - val_accuracy: 0.4737 - val_loss: 10.2302
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5440 - loss: 10.1843 - val_accuracy: 0.6637 - val_loss: 10.0538
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5939 - loss: 10.0165 - val_accuracy: 0.6388 - val_loss: 9.8921
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5748 - loss: 9.8590 - val_accuracy: 0.5938 - val_loss: 9.7430
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5955 - loss

[I 2024-08-04 13:50:59,172] Trial 0 finished with value: 0.6274999976158142 and parameters: {'num_layers': 3, 'filters_0': 81, 'filters_1': 126, 'filters_2': 62, 'kernel_size': 2, 'l1': 0.003188315471455216, 'l2': 2.1306421274251034e-06, 'dense_units': 188, 'dropout_rate': 0.2436894556788965, 'learning_rate': 1.5036844388351935e-05}. Best is trial 0 with value: 0.6274999976158142.


Trial 1 started with hyperparameters: num_layers=6, filters=[40, 67, 92, 32, 34, 55], kernel_size=2, l1=0.0037157522547798024, l2=0.00031460442046373686, dense_units=118, dropout_rate=0.14421927289369652, learning_rate=0.00013405854266477086
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 79ms/step - accuracy: 0.4940 - loss: 12.7731 - val_accuracy: 0.5525 - val_loss: 11.7009
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5104 - loss: 11.3581 - val_accuracy: 0.4800 - val_loss: 10.3526
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5312 - loss: 10.0321 - val_accuracy: 0.5788 - val_loss: 9.0940
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5342 - loss: 8.7961 - val_accuracy: 0.4938 - val_loss: 7.9261
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5083 - loss: 7.6502 - val_accuracy: 0.5625 - val_loss: 6.8469
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5419 - loss: 6.5936 - val_accuracy: 0.5050 - val_loss: 5.8570
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.48

...

Trial 28 started with hyperparameters: num_layers=3, filters=[46, 92, 36], kernel_size=2, l1=0.00036418951140005474, l2=0.0005864989581047493, dense_units=108, dropout_rate=0.3962471352461145, learning_rate=1.8836662706717208e-05
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4937 - loss: 1.3457

[I 2024-08-04 16:51:27,699] Trial 28 pruned. Trial was pruned at epoch 0.


Trial 29 started with hyperparameters: num_layers=3, filters=[64, 80, 75], kernel_size=2, l1=0.0027201470819170626, l2=1.4516792008647347e-05, dense_units=148, dropout_rate=0.43655419335065637, learning_rate=1.4983611953051604e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5113 - loss: 6.8007

[I 2024-08-04 16:52:57,506] Trial 29 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.6527500152587891
Params: 
    num_layers: 4
    filters_0: 50
    filters_1: 91
    filters_2: 73
    filters_3: 118
    kernel_size: 2
    l1: 0.0059151443315403895
    l2: 1.1259329056013594e-05
    dense_units: 142
    dropout_rate: 0.33221970808788714
    learning_rate: 1.6120646535662953e-05
Elapsed time: 11758.62 seconds


In [6]:
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import logging
import os
import time
import gzip
import skimage.transform
from optuna.integration import TFKerasPruningCallback

# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AS.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_strong/AS'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///as_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=30)  # Set to 30 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function


Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_strong/AS


[I 2024-08-04 16:53:47,831] A new study created in RDB with name: no-name-067d7dc7-68ad-43ea-b254-8bbaa3d81a1b


Trial 0 started with hyperparameters: num_layers=4, filters=[88, 38, 88, 106], kernel_size=3, l1=1.5022150089120748e-05, l2=0.0035801025371631446, dense_units=193, dropout_rate=0.2866844238417743, learning_rate=0.00016920857191232494
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - accuracy: 0.4980 - loss: 1.4448 - val_accuracy: 0.5175 - val_loss: 1.2840
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5155 - loss: 1.2405 - val_accuracy: 0.5200 - val_loss: 1.1213
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5080 - loss: 1.0901 - val_accuracy: 0.4888 - val_loss: 1.0035
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5203 - loss: 0.9798 - val_accuracy: 0.5813 - val_loss: 0.9162
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5418 - loss: 0.8997 - val_accuracy: 0.5100 - val_loss: 0.8571
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5257 - loss: 0.8457 - val_accuracy: 0.5050 - val_loss: 0.8200
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5750 - loss: 0.

[I 2024-08-04 17:08:01,942] Trial 0 finished with value: 0.6507499814033508 and parameters: {'num_layers': 4, 'filters_0': 88, 'filters_1': 38, 'filters_2': 88, 'filters_3': 106, 'kernel_size': 3, 'l1': 1.5022150089120748e-05, 'l2': 0.0035801025371631446, 'dense_units': 193, 'dropout_rate': 0.2866844238417743, 'learning_rate': 0.00016920857191232494}. Best is trial 0 with value: 0.6507499814033508.


Trial 1 started with hyperparameters: num_layers=5, filters=[121, 83, 93, 35, 127], kernel_size=2, l1=8.57767954844711e-06, l2=0.0018156400922245438, dense_units=162, dropout_rate=0.43753306747558995, learning_rate=1.4487334426505518e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 115ms/step - accuracy: 0.5112 - loss: 1.2640 - val_accuracy: 0.5288 - val_loss: 1.2556
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.4991 - loss: 1.2534 - val_accuracy: 0.4800 - val_loss: 1.2456
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.4995 - loss: 1.2433 - val_accuracy: 0.4875 - val_loss: 1.2350
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.5220 - loss: 1.2323 - val_accuracy: 0.5125 - val_loss: 1.2247
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.5141 - loss: 1.2222 - val_accuracy: 0.5163 - val_loss: 1.2148
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5341 - loss: 1.2124 - val_accuracy: 0.5200 - val_loss: 1.2053
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.5155 - los

...

Trial 28 started with hyperparameters: num_layers=6, filters=[106, 96, 85, 99, 93, 36], kernel_size=2, l1=0.0007205614464315885, l2=1.017806123650197e-05, dense_units=125, dropout_rate=0.26814687992250474, learning_rate=0.00035590581636867015
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - accuracy: 0.5232 - loss: 5.3721 - val_accuracy: 0.5175 - val_loss: 4.0915
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5165 - loss: 3.7168 - val_accuracy: 0.4800 - val_loss: 2.6893
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5007 - loss: 2.4060 - val_accuracy: 0.4725 - val_loss: 1.6583
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.4957 - loss: 1.4695 - val_accuracy: 0.5063 - val_loss: 1.0080
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5133 - loss: 0.9161 - val_accuracy: 0.4913 - val_loss: 0.7450
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.5015 - loss: 0.7304 - val_accuracy: 0.5050 - val_loss: 0.7058
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.4846 -

[I 2024-08-04 20:37:13,448] Trial 28 finished with value: 0.5 and parameters: {'num_layers': 6, 'filters_0': 106, 'filters_1': 96, 'filters_2': 85, 'filters_3': 99, 'filters_4': 93, 'filters_5': 36, 'kernel_size': 2, 'l1': 0.0007205614464315885, 'l2': 1.017806123650197e-05, 'dense_units': 125, 'dropout_rate': 0.26814687992250474, 'learning_rate': 0.00035590581636867015}. Best is trial 0 with value: 0.6507499814033508.


Trial 29 started with hyperparameters: num_layers=5, filters=[119, 52, 102, 82, 126], kernel_size=3, l1=1.5488173218113703e-05, l2=0.0014269498193011146, dense_units=110, dropout_rate=0.32992348013633777, learning_rate=5.1802745298826076e-05
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4970 - loss: 1.2846

[I 2024-08-04 20:38:49,656] Trial 29 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.6507499814033508
Params: 
    num_layers: 4
    filters_0: 88
    filters_1: 38
    filters_2: 88
    filters_3: 106
    kernel_size: 3
    l1: 1.5022150089120748e-05
    l2: 0.0035801025371631446
    dense_units: 193
    dropout_rate: 0.2866844238417743
    learning_rate: 0.00016920857191232494
Elapsed time: 13502.07 seconds


In [7]:
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import logging
import os
import time
import gzip
import skimage.transform
from optuna.integration import TFKerasPruningCallback

# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AW.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_weak/AW'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///aw_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=30)  # Set to 30 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function


Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_weak/AW


[I 2024-08-04 20:43:01,424] A new study created in RDB with name: no-name-e83c06ce-ce7b-44ee-a988-e1ccf1a89758


Trial 0 started with hyperparameters: num_layers=3, filters=[32, 112, 80], kernel_size=2, l1=5.6564905040331065e-06, l2=0.0006596203709234314, dense_units=91, dropout_rate=0.22209327106425047, learning_rate=2.7822404420087316e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.5095 - loss: 0.8014 - val_accuracy: 0.5387 - val_loss: 0.7875
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6045 - loss: 0.7792 - val_accuracy: 0.6975 - val_loss: 0.7487
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6771 - loss: 0.7451 - val_accuracy: 0.6950 - val_loss: 0.7143
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6721 - loss: 0.7183 - val_accuracy: 0.7337 - val_loss: 0.6788
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6924 - loss: 0.6908 - val_accuracy: 0.7237 - val_loss: 0.6620
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7284 - loss: 0.6612 - val_accuracy: 0.7275 - val_loss: 0.6671
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7081 - loss: 0.6627 

[I 2024-08-04 20:56:54,247] Trial 0 finished with value: 0.7494999766349792 and parameters: {'num_layers': 3, 'filters_0': 32, 'filters_1': 112, 'filters_2': 80, 'kernel_size': 2, 'l1': 5.6564905040331065e-06, 'l2': 0.0006596203709234314, 'dense_units': 91, 'dropout_rate': 0.22209327106425047, 'learning_rate': 2.7822404420087316e-05}. Best is trial 0 with value: 0.7494999766349792.


Trial 1 started with hyperparameters: num_layers=3, filters=[77, 37, 98], kernel_size=2, l1=1.372647917189269e-06, l2=0.0047751555805138944, dense_units=172, dropout_rate=0.1236472638677225, learning_rate=1.0186907345142566e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.5103 - loss: 1.1950 - val_accuracy: 0.4825 - val_loss: 1.1860
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5583 - loss: 1.1791 - val_accuracy: 0.7387 - val_loss: 1.1625
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6769 - loss: 1.1558 - val_accuracy: 0.5888 - val_loss: 1.1503
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6569 - loss: 1.1375 - val_accuracy: 0.7400 - val_loss: 1.1081
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7037 - loss: 1.1104 - val_accuracy: 0.7375 - val_loss: 1.0846
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7334 - loss: 1.0798 - val_accuracy: 0.7325 - val_loss: 1.0702
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7172 - loss: 1.0651 - 

...

Trial 28 started with hyperparameters: num_layers=3, filters=[96, 48, 118], kernel_size=2, l1=2.593774609648388e-06, l2=0.0001665345041415093, dense_units=197, dropout_rate=0.29508920580027626, learning_rate=4.151554702128399e-05
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5074 - loss: 0.7296

[I 2024-08-04 23:42:08,657] Trial 28 pruned. Trial was pruned at epoch 0.


Trial 29 started with hyperparameters: num_layers=5, filters=[86, 39, 94, 107, 79], kernel_size=2, l1=0.00030344891195144437, l2=0.0005471693294126225, dense_units=225, dropout_rate=0.44225500363817927, learning_rate=0.00012994004948262334
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.4940 - loss: 2.2494 - val_accuracy: 0.4825 - val_loss: 2.1026
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5460 - loss: 2.0551 - val_accuracy: 0.7113 - val_loss: 1.9151
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6433 - loss: 1.8620 - val_accuracy: 0.7063 - val_loss: 1.6658
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7064 - loss: 1.6231 - val_accuracy: 0.7325 - val_loss: 1.4950
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7119 - loss: 1.4873 - val_accuracy: 0.7400 - val_loss: 1.3936
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7357 - loss: 1.3727 - val_accuracy: 0.7262 - val_loss: 1.3215
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7341 - lo

[I 2024-08-04 23:54:45,786] Trial 29 pruned. 


Best trial:
Value: 0.7509999871253967
Params: 
    num_layers: 3
    filters_0: 79
    filters_1: 79
    filters_2: 125
    kernel_size: 2
    l1: 1.4238915610455433e-06
    l2: 0.0011892049877332892
    dense_units: 227
    dropout_rate: 0.2605345852243278
    learning_rate: 2.5834311151381096e-05
Elapsed time: 11504.60 seconds
